# **PREPARANDO O AMBIENTE**

In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
import requests
import json
import openai
import pandas as pd

# **EXTRAÇÃO**

## **CRIANDO LISTA DE IDS**

In [ ]:
ids = pd.read_csv('SDW2023.csv')

In [ ]:
ids.head()

,UserID
0,199
1,200
2,201
3,202


In [ ]:
id_lista = ids['UserID'].tolist() #Transforma os IDs numa lista
id_lista

[199, 200, 201, 202]

## **CONSUMINDO A API**

In [ ]:
api_url = 'https://sdw-2023-prd.up.railway.app'

In [ ]:
def get_user(id):
  response = requests.get(f'{api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

In [ ]:
get_user(199)

## **CRIANDO A LISTA DE CLIENTES**

In [ ]:
users = [user for id in id_lista if (user := get_user(id))is not None] #
print(json.dumps(users, indent=1))

{'id': 202,
 'name': 'Pedro Costa',
 'account': {'id': 207,
  'number': '8901-2',
  'agency': '0004-6',
  'balance': 2000.0,
  'limit': 1000.0},
 'card': {'id': 195, 'number': '5678-9012-3456-7890', 'limit': 2500.0},
 'features': [],
 'news': [{'id': 608,
   'icon': 'https://images.genius.com/31fac7e2bef0ff83f25dbd8077863794.1000x1000x1.jpg',
   'description': 'Eduardo esteve aqui'},
  {'id': 609,
   'icon': 'https://images.genius.com/31fac7e2bef0ff83f25dbd8077863794.1000x1000x1.jpg',
   'description': 'Eduardo esteve aqui'}]}

# **TRANSFORMAÇÃO**

In [ ]:
# Integrando com a openai
openai_api_key = 'chave_api_aqui'
openai.api_key = openai_api_key

In [ ]:
#Gerar mensagem personalizada com ajuda do GPT 3.5-Turbo

def gerar_mensagem_ia(user):
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

In [ ]:
for user in users:
  mensagem = gerar_mensagem_ia(user)
  print(mensagem)
  user['mensagem'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": mensagem
  })

RateLimitError: ignored

# **CARREGAMENTO**

In [ ]:
def update_user(user):
  response = requests.put(f"{api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

In [ ]:
for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User João Silva updated? True!
User Maria Santos updated? True!
User Ana Oliveira updated? True!
User Pedro Costa updated? True!
